#### **Path + Partições:**

In [ ]:
#--------------------#
# setup
#--------------------#

path_dados = "/Users/rdahis/Downloads/dados_SICONFI"
path_queries = "/Users/rdahis/Dropbox/BD/queries-basedosdados/models/br_me_siconfi"
# requirements: pandas, openpyxl

import pandas as pd
import os
import glob

pd.options.mode.chained_assignment = None  # default='warn'

LAST_YEAR = 2023

In [ ]:
# cria pastas de receita e despesas por ano e UF

ufs = ["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO",
       "MA", "MG", "MS", "MT", "PA", "PB", "PE", "PI", "PR",
       "RJ", "RN", "RO", "RR", "SC", "SE", "RS", "SP", "TO"]

for i in [*range(1989, LAST_YEAR+1)]:
  for uf in ufs:

    path = os.path.join(path_dados, 'output/municipio_receitas_orcamentarias/ano={}/sigla_uf={}'.format(i, uf))
    if not os.path.exists(path):
      os.makedirs(path)

    path = os.path.join(path_dados, 'output/municipio_despesas_orcamentarias/ano={}/sigla_uf={}'.format(i, uf))
    if not os.path.exists(path):
      os.makedirs(path)

for i in [*range(1996, LAST_YEAR+1)]:
  for uf in ufs:

    path = os.path.join(path_dados, 'output/municipio_despesas_funcao/ano={}/sigla_uf={}'.format(i, uf))
    if not os.path.exists(path):
      os.makedirs(path)

for i in [*range(1998, LAST_YEAR+1)]:
  for uf in ufs:

    path = os.path.join(path_dados, 'output/municipio_balanco_patrimonial/ano={}/sigla_uf={}'.format(i, uf))
    if not os.path.exists(path):
      os.makedirs(path)

for i in [*range(2013, LAST_YEAR+1)]:
  for uf in ufs:
    for table in ['uf_receitas_orcamentarias', 'uf_despesas_orcamentarias', 'uf_despesas_funcao']:
      path = os.path.join(path_dados, 'output/{}/ano={}/sigla_uf={}'.format(table, i, uf))
      if not os.path.exists(path):
        os.makedirs(path)


In [ ]:
ordem = {
    'municipio': ['ano', 'sigla_uf', 'id_municipio', 'estagio', 'portaria', 'conta', 'estagio_bd', 'id_conta_bd', 'conta_bd', 'valor'],
    'uf':        ['ano', 'sigla_uf', 'id_uf',        'estagio', 'portaria', 'conta', 'estagio_bd', 'id_conta_bd', 'conta_bd', 'valor']
}

#### **Contas - Joins:**

In [ ]:
df_comp_municipio = pd.read_excel(os.path.join(path_queries, 'code/compatibilizacao/municipio.xlsx'), dtype='string')

df_comp_receitas = pd.read_excel(os.path.join(path_queries, 'code/compatibilizacao/receitas_orcamentarias.xlsx'), dtype='string')
df_comp_receitas = df_comp_receitas.fillna('')

df_comp_despesas = pd.read_excel(os.path.join(path_queries, 'code/compatibilizacao/despesas_orcamentarias.xlsx'), dtype='string')
df_comp_despesas = df_comp_despesas.fillna('')

df_comp_despesas_funcao = pd.read_excel(os.path.join(path_queries, 'code/compatibilizacao/despesas_funcao.xlsx'), dtype='string')
df_comp_despesas_funcao = df_comp_despesas_funcao.fillna('')

df_comp_balanco_patrimonial = pd.read_excel(os.path.join(path_queries, 'code/compatibilizacao/balanco_patrimonial.xlsx'), dtype='string')
df_comp_balanco_patrimonial = df_comp_balanco_patrimonial.fillna('')

#### **Receitas Orçamentarias**:


In [ ]:
for ano in [*range(1989, LAST_YEAR+1)]:

  #-------------------------------#
  # municipio
  #-------------------------------#

  df = pd.DataFrame(columns=ordem['municipio'])

  arquivos = [
      arquivo
        for arquivo in glob.iglob(os.path.join(path_dados, 'input/municipio/quadro*'))
          if int(arquivo[-11:-7]) == ano and
           ((ano >= 1989 and ano <= 1996 and int(arquivo[-6:-5]) <= 3) or
            (ano >= 1997 and ano <= 2012 and int(arquivo[-6:-5]) == 1)
           )
  ]

  if ano >= 2013:
    anexo = 'finbra_MUN_ReceitasOrcamentarias(AnexoI-C)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivos.append(os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo)))

  for arquivo in arquivos:

    if (ano >= 1989 and ano <= 2012):
      
      df_dados = pd.read_excel(arquivo, dtype='string', na_values="")

      if (ano >= 1989 and ano <= 1997):
        
        df_dados['municipio_original'] = df_dados['municipio_original'].str.strip()
        df_dados = pd.merge(df_dados, df_comp_municipio,
                            how='left',
                            left_on =['municipio_original', 'sigla_uf'],
                            right_on=['municipio_original', 'sigla_uf'])
      
      if (ano >= 1998 and ano <= 2012):
        
        df_dados['length'] = df_dados['CD_MUN'].str.len()
        df_dados['id_municipio_6'] = '0'
        df_dados.loc[df_dados['length'] == 1, 'id_municipio_6'] = df_dados['CD_UF'] + '000' + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 2, 'id_municipio_6'] = df_dados['CD_UF'] + '00'  + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 3, 'id_municipio_6'] = df_dados['CD_UF'] + '0'   + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 4, 'id_municipio_6'] = df_dados['CD_UF']         + df_dados['CD_MUN']
        
        df_comp_municipio_reduzido                   = df_comp_municipio[['id_municipio', 'sigla_uf']]
        df_comp_municipio_reduzido['id_municipio_6'] = df_comp_municipio_reduzido['id_municipio'].str[0:6]

        df_dados = pd.merge(
          df_dados, df_comp_municipio_reduzido,
          how='left',
          left_on =['id_municipio_6'],
          right_on=['id_municipio_6']
        )

        df_dados = df_dados.drop(['length', 'id_municipio_6'], axis = 1)

      df_dados.drop_duplicates(subset=['id_municipio'], keep ='first', inplace = True)

      if (ano >= 1989 and ano <= 1997): id_vars = ['id_municipio', 'sigla_uf', 'municipio_auxiliar', 'municipio_original']
      if (ano >= 1998 and ano <= 2012): id_vars = ['id_municipio', 'sigla_uf',                                             'CD_UF', 'CD_MUN'                  ]

      value_vars = df_dados.drop(id_vars, axis = 1).columns
      df_dados = pd.melt(df_dados, id_vars = id_vars, value_vars = value_vars, var_name='conta', value_name='valor')

      if (ano >= 1989 and ano <= 1997):
        to_drop = ['municipio_auxiliar', 'municipio_original']
      else:
        to_drop = ['CD_UF', 'CD_MUN']
      df_dados.drop(to_drop, axis=1, inplace=True)

    if ano >= 2013:

      df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')
      !rm "{arquivo}"

      df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
      df_dados.columns = ['id_municipio', 'sigla_uf', 'estagio', 'conta', 'valor']

      if (ano >= 2013 and ano <= 2017):

        df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split(' -', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
        df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('- ', 1)[1] if x[0].isnumeric() else x).astype('string').str.strip()

      if ano >= 2018: # TODO: double check if it should be 2017 here

        df_dados['portaria'] = df_dados['conta'].apply(lambda x: x[:14] if x[0].isnumeric() else "").astype('string').str.strip()
        df_dados['conta']    = df_dados['conta'].apply(lambda x: x[16:] if (x[0].isnumeric() and '0 -' in x) else x[15:] if x[0].isnumeric() else x).astype('string').str.strip()

      df_dados['conta'] = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
      df_dados['valor'] = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    if (ano <= 2012): chaves = ['ano',                        'conta']
    if (ano >= 2013): chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_receitas,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_municipio', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    if ano >= 1990 and ano <= 1993: # to fix issue pointed out at https://github.com/basedosdados/mais/issues/1426
      df_dados['valor'] = df_dados.apply(lambda row: 1000 * row.valor, axis=1)

    if ano <= 1992:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/(2.75*(1000**2)), axis=1)
    elif ano == 1993:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/(2.75*1000), axis=1)

    df_dados = df_dados[ordem['municipio']]
    
    df = pd.concat([df, df_dados], ignore_index= True)

  #----------------#
  # particiona
  #----------------#

  print("Particionando {}".format(ano))

  for uf in ufs:
    df_partition = df.loc[df['sigla_uf'] == uf]
    df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
    partition_path = os.path.join(path_dados, 'output/municipio_receitas_orcamentarias/ano={}/sigla_uf={}/municipio_receitas_orcamentarias.csv'.format(ano,uf))
    df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')

  #-------------------------------#
  # uf
  #-------------------------------#

  if ano >= 2013:
    df = pd.DataFrame(columns=ordem['uf'])
    anexo = 'finbra_ESTDF_ReceitasOrcamentarias(AnexoI-C)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivo = os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo))
    df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')
    !rm "{arquivo}"

    df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
    df_dados.columns = ['id_uf', 'sigla_uf', 'estagio', 'conta', 'valor']

    if (ano >= 2013 and ano <= 2017): # TODO: double check if it should be 2018 here

      df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split(' -', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('- ', 1)[1] if x[0].isnumeric() else x).astype('string').str.strip()

    else:

      df_dados['portaria'] = df_dados['conta'].apply(lambda x: x[:14] if x[0].isnumeric() else "").astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x[16:] if (x[0].isnumeric() and '0 -' in x) else x[15:] if x[0].isnumeric() else x).astype('string').str.strip()

    df_dados['conta'] = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
    df_dados['valor'] = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_receitas,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_uf', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    df_dados = df_dados[ordem['uf']]
    df = pd.concat([df, df_dados], ignore_index= True)

    #----------------#
    # particiona
    #----------------#

    print("Particionando {}".format(ano))

    for uf in ufs:
      df_partition = df.loc[df['sigla_uf'] == uf]
      df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
      partition_path = os.path.join(path_dados, 'output/uf_receitas_orcamentarias/ano={}/sigla_uf={}/uf_receitas_orcamentarias.csv'.format(ano,uf))
      df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')



#### **Despesas Orçamentárias:**

In [ ]:
for ano in [*range(1989, LAST_YEAR+1)]:

  #-------------------------------#
  # municipio
  #-------------------------------#

  df = pd.DataFrame(columns=ordem['municipio'])

  arquivos = [
      arquivo
        for arquivo in glob.iglob(os.path.join(path_dados, 'input/municipio/quadro*'))
          if int(arquivo[-11:-7]) == ano and
           ((ano >= 1989 and ano <= 1996 and      int(arquivo[-6:-5]) == 4) or
            (ano == 1997 and                      int(arquivo[-6:-5]) == 3) or
            (ano >= 1998 and ano <= 2008 and      int(arquivo[-6:-5]) == 2) or
            (ano >= 2009 and ano <= 2012 and 2 <= int(arquivo[-6:-5]) <= 4)
           )
  ]

  if ano >= 2013:
    anexo = 'finbra_MUN_DespesasOrcamentarias(AnexoI-D)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivos.append(os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo)))

  for arquivo in arquivos:

    if ano >= 2013:
      tipo_arquivo = 0
    else:
      tipo_arquivo = int(arquivo[-6:-5])

    if (ano >= 1989 and ano <= 2012):

      df_dados = pd.read_excel(arquivo, dtype='string', na_values="")

      if (ano >= 1989 and ano <= 1997):
        
        df_dados['municipio_original'] = df_dados['municipio_original'].str.strip()
        df_dados = pd.merge(df_dados, df_comp_municipio,
                            how='left',
                            left_on =['municipio_original', 'sigla_uf'],
                            right_on=['municipio_original', 'sigla_uf'])
      
      if (ano >= 1998 and ano <= 2012):
        
        df_dados['length'] = df_dados['CD_MUN'].str.len()
        df_dados['id_municipio_6'] = '0'
        df_dados.loc[df_dados['length'] == 1, 'id_municipio_6'] = df_dados['CD_UF'] + '000' + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 2, 'id_municipio_6'] = df_dados['CD_UF'] + '00'  + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 3, 'id_municipio_6'] = df_dados['CD_UF'] + '0'   + df_dados['CD_MUN']
        df_dados.loc[df_dados['length'] == 4, 'id_municipio_6'] = df_dados['CD_UF']         + df_dados['CD_MUN']
        
        df_comp_municipio_reduzido                   = df_comp_municipio[['id_municipio', 'sigla_uf']]
        df_comp_municipio_reduzido['id_municipio_6'] = df_comp_municipio_reduzido['id_municipio'].str[0:6]

        df_dados = pd.merge(
          df_dados, df_comp_municipio_reduzido,
          how='left',
          left_on =['id_municipio_6'],
          right_on=['id_municipio_6']
        )

        df_dados = df_dados.drop(['length', 'id_municipio_6'], axis = 1)
      
      df_dados.drop_duplicates(subset=['id_municipio'], keep ='first', inplace = True)

      df_dados.to_csv("~/Downloads/siconfi.csv", index=False, encoding='utf-8', na_rep='')

      if (ano >= 1989 and ano <= 1997): id_vars = ['id_municipio', 'sigla_uf', 'municipio_auxiliar', 'municipio_original'                  ]
      if (ano >= 1998 and ano <= 2012): id_vars = ['id_municipio', 'sigla_uf',                                            'CD_UF', 'CD_MUN']

      value_vars = df_dados.drop(id_vars, axis = 1).columns
      df_dados = pd.melt(df_dados, id_vars = id_vars, value_vars = value_vars, var_name='conta', value_name='valor') # esse passo introduz muitas linhas com valor=0 em células que antes não existiam

      if (ano >= 2009 and ano <= 2012):
        if tipo_arquivo == 2:
          df_dados['estagio'] = 'Despesas Empenhadas'
        elif tipo_arquivo == 3:
          df_dados['estagio'] = 'Despesas Liquidadas'
        elif tipo_arquivo == 4:
          df_dados['estagio'] = 'Despesas Pagas'

      if (ano >= 2004 and ano <= 2012):
        df_dados.loc[(df_dados['conta'] =="ADAD  Inden e Restituições" ), 'conta'] = "ADAD Inden e Restituições"
        df_dados.loc[(df_dados['conta'] =="IFAD  Inden e Restituições" ), 'conta'] = "IFAD Inden e Restituições"

      if (ano >= 1989 and ano <= 1997):
        to_drop = ['municipio_auxiliar', 'municipio_original']
      else:
        to_drop = ['CD_UF', 'CD_MUN']
      df_dados.drop(to_drop, axis=1, inplace=True)

    if (ano >= 2013):

      df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')

      df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
      df_dados.columns = ['id_municipio', 'sigla_uf', 'estagio', 'conta', 'valor']

      df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split('-', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('-', 1)[1] if x[0].isnumeric() else  x).astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
      df_dados['valor']    = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    if (ano <= 2008                ): chaves = ['ano',                        'conta']
    if (ano >= 2009 and ano <= 2012): chaves = ['ano', 'estagio',             'conta']
    if (ano >= 2013                ): chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_despesas,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_municipio', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    # conversão de moedas para Real
    if ano >= 1990 and ano <= 1993: # to fix issue pointed out at https://github.com/basedosdados/mais/issues/1426
      df_dados['valor'] = df_dados.apply(lambda row: 1000 * row.valor, axis=1)

    if ano <= 1992:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/((1000**2)*2.75), axis=1)
    elif ano == 1993:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/(1000*2.75), axis=1)

    df_dados = df_dados[ordem['municipio']]
    df = pd.concat([df, df_dados], ignore_index= True)

  #----------------#
  # particiona
  #----------------#

  print("Particionando {}.".format(ano))

  for uf in ufs:
    df_partition = df.loc[df['sigla_uf'] == uf]
    df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
    partition_path = os.path.join(path_dados, 'output/municipio_despesas_orcamentarias/ano={}/sigla_uf={}/municipio_despesas_orcamentarias.csv'.format(ano,uf))
    df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')

  #-------------------------------#
  # uf
  #-------------------------------#

  if ano >= 2013:

    df = pd.DataFrame(columns=ordem['uf'])

    anexo = 'finbra_ESTDF_DespesasOrcamentarias(AnexoI-D)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivo = os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo))
    df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')
    !rm "{arquivo}"

    df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
    df_dados.columns = ['id_uf', 'sigla_uf', 'estagio', 'conta', 'valor']

    df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split('-', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
    df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('-', 1)[1] if x[0].isnumeric() else  x).astype('string').str.strip()
    df_dados['conta']    = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
    df_dados['valor']    = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_despesas,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_uf', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    df_dados = df_dados[ordem['uf']]
    df = pd.concat([df, df_dados], ignore_index= True)

    #----------------#
    # particiona
    #----------------#

    print("Particionando {}".format(ano))

    for uf in ufs:
      df_partition = df.loc[df['sigla_uf'] == uf]
      df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
      partition_path = os.path.join(path_dados, 'output/uf_despesas_orcamentarias/ano={}/sigla_uf={}/uf_despesas_orcamentarias.csv'.format(ano,uf))
      df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')


#### **Despesa por Função**:


In [ ]:
for ano in [*range(1996, LAST_YEAR+1)]:

  #-------------------------------#
  # municipio
  #-------------------------------#

  df = pd.DataFrame(columns=ordem['municipio'])

  arquivos = [
      arquivo
        for arquivo in glob.iglob(os.path.join(path_dados, 'input/municipio/quadro*'))
          if int(arquivo[-11:-7]) == ano and
           ((ano >= 1996 and ano <= 2012 and int(arquivo[-6:-5]) == 5))
  ]
  
  if ano >= 2013:
    anexo = 'finbra_MUN_DespesasporFuncao(AnexoI-E)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivos.append(os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo)))

  for arquivo in arquivos:

    if ano <= 2012:
      
      tipo_arquivo = int(arquivo[-6:-5])
      df_dados = pd.read_excel(arquivo, dtype='string', na_values="")
      
      if (ano >= 1996 and ano <= 1997):
        
        df_dados['municipio_original'] = df_dados['municipio_original'].str.strip()
        df_dados = pd.merge(df_dados, df_comp_municipio,
                            how='left',
                            left_on =['municipio_original', 'sigla_uf'],
                            right_on=['municipio_original', 'sigla_uf'])

      elif (ano >= 1998 and ano <= 2012):

        df_dados['id_municipio_6'] = '0'

        if ano >= 1998 and ano <= 2003:
          df_dados['length'] = df_dados['CD_MUN'].str.len()
          df_dados.loc[df_dados['length'] == 1, 'id_municipio_6'] = df_dados['CD_UF'] + '000' + df_dados['CD_MUN']
          df_dados.loc[df_dados['length'] == 2, 'id_municipio_6'] = df_dados['CD_UF'] + '00'  + df_dados['CD_MUN']
          df_dados.loc[df_dados['length'] == 3, 'id_municipio_6'] = df_dados['CD_UF'] + '0'   + df_dados['CD_MUN']
          df_dados.loc[df_dados['length'] == 4, 'id_municipio_6'] = df_dados['CD_UF']         + df_dados['CD_MUN']
        else:
          df_dados['length'] = df_dados['Cod Mun'].str.len()
          df_dados.loc[df_dados['length'] == 1, 'id_municipio_6'] = df_dados['UF'] + '000' + df_dados['Cod Mun']
          df_dados.loc[df_dados['length'] == 2, 'id_municipio_6'] = df_dados['UF'] + '00'  + df_dados['Cod Mun']
          df_dados.loc[df_dados['length'] == 3, 'id_municipio_6'] = df_dados['UF'] + '0'   + df_dados['Cod Mun']
          df_dados.loc[df_dados['length'] == 4, 'id_municipio_6'] = df_dados['UF']         + df_dados['Cod Mun']
        
        df_comp_municipio_reduzido                   = df_comp_municipio[['id_municipio', 'sigla_uf']]
        df_comp_municipio_reduzido['id_municipio_6'] = df_comp_municipio_reduzido['id_municipio'].str[0:6]

        df_dados = pd.merge(
          df_dados, df_comp_municipio_reduzido,
          how='left',
          left_on =['id_municipio_6'],
          right_on=['id_municipio_6']
        )

        df_dados = df_dados.drop(['length', 'id_municipio_6'], axis = 1)
      
      df_dados.drop_duplicates(subset=['id_municipio'], keep ='first', inplace = True)

      if (ano >= 1996 and ano <= 1997):
        id_vars = ['id_municipio', 'sigla_uf', 'municipio_auxiliar', 'municipio_original']
        to_drop = ['municipio_auxiliar', 'municipio_original']
      elif (ano >= 1998 and ano <= 2003):
        id_vars = ['id_municipio', 'sigla_uf', 'CD_UF', 'CD_MUN'] #, 'municipio_auxiliar', 'municipio_original', 'CdUF', 'CdMun', 'Populacao']
        to_drop = ['CD_UF', 'CD_MUN']
      else:
        id_vars = ['id_municipio', 'sigla_uf', 'UF', 'Cod Mun'] #, 'municipio_auxiliar', 'municipio_original', 'CdUF', 'CdMun', 'Populacao']
        to_drop = ['UF', 'Cod Mun']
      
      value_vars = df_dados.drop(id_vars, axis = 1).columns
      df_dados = pd.melt(df_dados, id_vars = id_vars, value_vars = value_vars, var_name='conta', value_name='valor')
      df_dados.drop(to_drop, axis=1, inplace=True)

    if (ano >= 2013):

      tipo_arquivo = 0
      df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')
      !rm "{arquivo}"

      df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
      df_dados.columns = ['id_municipio', 'sigla_uf', 'estagio', 'conta', 'valor']

      df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split('-', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
      df_dados['portaria'] = df_dados['portaria'].str.lstrip('0')
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('-', 1)[1] if x[0].isnumeric() else  x).astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
      df_dados['valor']    = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    if (ano <= 2012): chaves = ['ano',                        'conta']
    if (ano >= 2013): chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_despesas_funcao,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_municipio', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    if ano >= 1990 and ano <= 1993: # to fix issue pointed out at https://github.com/basedosdados/mais/issues/1426
      df_dados['valor'] = df_dados.apply(lambda row: 1000 * row.valor, axis=1)

    if ano <= 1992:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/((1000**2)*2.75), axis=1)
    elif ano == 1993:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/(1000*2.75), axis=1)

    df_dados = df_dados[ordem['municipio']]

    df = pd.concat([df, df_dados], ignore_index= True)

  #----------------#
  # particiona
  #----------------#
  
  #df.to_csv("~/Downloads/municipio_despesas_funcao_{}.csv".format(ano), index=False, encoding='utf-8', na_rep='')

  print("Particionando {}".format(ano))

  #for uf in ufs:
  #  df_partition = df.loc[df['sigla_uf'] == uf]
  #  df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
  #  partition_path = os.path.join(path_dados, 'output/municipio_despesas_funcao/ano={}/sigla_uf={}/municipio_despesas_funcao.csv'.format(ano,uf))
  #  df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')

  #-------------------------------#
  # uf
  #-------------------------------#

  if ano >= 2013:

    df = pd.DataFrame(columns=ordem['uf'])

    anexo = 'finbra_ESTDF_DespesasporFuncao(AnexoI-E)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivo = os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo))
    df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')
    !rm "{arquivo}"

    df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
    df_dados.columns = ['id_uf', 'sigla_uf', 'estagio', 'conta', 'valor']

    df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split('-', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
    df_dados['portaria'] = df_dados['portaria'].str.lstrip('0')
    df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('-', 1)[1] if x[0].isnumeric() else  x).astype('string').str.strip()
    df_dados['conta']    = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
    df_dados['valor']    = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_despesas_funcao,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_uf', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    df_dados = df_dados[ordem['uf']]

    df = pd.concat([df, df_dados], ignore_index= True)

    #----------------#
    # particiona
    #----------------#

    print("Particionando {}".format(ano))

    for uf in ufs:
      df_partition = df.loc[df['sigla_uf'] == uf]
      df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
      partition_path = os.path.join(path_dados, 'output/uf_despesas_funcao/ano={}/sigla_uf={}/uf_despesas_funcao.csv'.format(ano,uf))
      df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')


#### **Balanço Patrimonial (a terminar)**

In [ ]:
for ano in [*range(2004, LAST_YEAR+1)]:

  df = pd.DataFrame(columns=ordem['municipio'])

  arquivos = [
      arquivo
        for arquivo in glob.iglob(os.path.join(path_dados, 'input/municipio/quadro*'))
          if int(arquivo[-11:-7]) == ano and
           (
              (ano >= 1998 and ano <= 2003 and 5 <= int(arquivo[-6:-5]) <= 6) or
              (ano >= 2004 and ano <= 2012 and 6 <= int(arquivo[-6:-5]) <= 7)
           )
  ]

  if ano >= 2013:
    anexo = 'finbra_MUN_BalancoPatrimonialDCA(AnexoI-AB)'
    zip = os.path.join(path_dados, 'input/{}/{}.zip'.format(anexo, ano))
    folder = os.path.join(path_dados, 'input/{}'.format(anexo))
    !unzip -o "{zip}" -d "{folder}"
    arquivos.append(os.path.join(path_dados, 'input/{}/finbra.csv'.format(anexo)))

  for arquivo in arquivos:

    if ano >= 2013:
      tipo_arquivo = 0
    else:
      tipo_arquivo = int(arquivo[-6:-5])

    if (ano >= 1998 and ano <= 2012):

      # TODO: adapt code below to balanco_patrimonial
      df_dados = pd.read_excel(arquivo, dtype='string', na_values="")
      df_dados['municipio_original'] = df_dados['municipio_original'].str.strip()

      df_dados = pd.merge(df_dados, df_comp_municipio,
                          how='left',
                          left_on =['municipio_original', 'sigla_uf'],
                          right_on=['municipio_original', 'sigla_uf'])
      df_dados.drop_duplicates(subset=['id_municipio'], keep ='first', inplace = True)

      id_vars = ['id_municipio', 'sigla_uf', 'municipio_auxiliar', 'municipio_original', 'CdUF', 'CdMun', 'Populacao']
      value_vars = df_dados.drop(id_vars, axis = 1).columns
      df_dados = pd.melt(df_dados, id_vars = id_vars, value_vars = value_vars, var_name='conta', value_name='valor')

      to_drop = ['municipio_auxiliar', 'municipio_original', 'CdUF', 'CdMun', 'Populacao']
      df_dados.drop(to_drop, axis=1, inplace=True)

    if (ano >= 2013):

      df_dados = pd.read_csv(arquivo, skiprows=3, encoding='latin1', sep=';', dtype= 'string')

      df_dados = df_dados.drop(['Instituição', 'População', 'Identificador da Conta'], axis =1)
      df_dados.columns = ['id_municipio', 'sigla_uf', 'estagio', 'conta', 'valor']

      df_dados['portaria'] = df_dados['conta'].apply(lambda x: x.split('-', 1)[0] if x[0].isnumeric() else "").astype('string').str.strip()
      df_dados['portaria'] = df_dados['portaria'].str.lstrip('0')
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.split('-', 1)[1] if x[0].isnumeric() else  x).astype('string').str.strip()
      df_dados['conta']    = df_dados['conta'].apply(lambda x: x.replace('¿', '-'))
      df_dados['valor']    = df_dados['valor'].apply(lambda x: x.replace(",", "."))

    df_dados['ano'] = ano
    df_dados['ano'] = df_dados['ano'].astype('string')

    if (ano <= 2012): chaves = ['ano',                        'conta']
    if (ano >= 2013): chaves = ['ano', 'estagio', 'portaria', 'conta']
    df_dados = pd.merge(df_dados, df_comp_despesas_funcao,
                        how='left',
                        left_on = chaves,
                        right_on= chaves)
    #df.drop_duplicates(subset=['id_municipio', 'conta', 'valor'], keep='first', inplace=True)

    df_dados['conta'] = df_dados['conta'].astype('string')
    df_dados = df_dados.fillna('')

    df_dados['valor'] = pd.to_numeric(df_dados['valor']).astype('float')

    if ano >= 1990 and ano <= 1993: # to fix issue pointed out at https://github.com/basedosdados/mais/issues/1426
      df_dados['valor'] = df_dados.apply(lambda row: 1000 * row.valor, axis=1)

    if ano <= 1992:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/((1000**2)*2.75), axis=1)
    elif ano == 1993:
      df_dados['valor'] = df_dados.apply(lambda row: (row.valor)/(1000*2.75), axis=1)

    df_dados = df_dados[ordem['municipio']]

    df = pd.concat([df, df_dados], ignore_index= True)

  #----------------#
  # particiona
  #----------------#

  print("Particionando {}".format(ano))

  for uf in ufs:
    df_partition = df.loc[df['sigla_uf'] == uf]
    df_partition.drop(['sigla_uf', 'ano'], axis=1, inplace=True)
    partition_path = os.path.join(path_dados, 'output/municipio_despesas_funcao/ano={}/sigla_uf={}/municipio_despesas_funcao.csv'.format(ano,uf))
    df_partition.to_csv(partition_path, index=False, encoding='utf-8', na_rep='')
